In [1]:
print("Welcome")
import requests
import datetime
import yaml #pyyaml
import numpy as np
import dataframe_image as dfi #https://randomds.com/2021/12/23/visualize-and-save-full-pandas-dataframes-as-images/
from datetime import datetime
from dateutil.relativedelta import relativedelta
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows
# looks useful: https://geo.rocks/post/python-to-exe/ from https://stackoverflow.com/questions/55741607/is-it-possible-to-generate-an-executable-exe-of-a-jupyter-notebook
# Merging in Pandas: https://stackoverflow.com/questions/53645882/pandas-merging-101

# Hold credentials in .yaml?
# Use python to hit call some code in an excel file? MAybe use batch as an intermediary?

# Pandas Pivot Table Filter:
# https://stackoverflow.com/questions/43235930/adding-filter-to-pandas-pivot-table
# Pivot table basics: https://builtin.com/data-science/pandas-pivot-tables



Welcome


In [2]:
# Imports username & security_token from setup.yaml
# get security token from... https://tigertext.lightning.force.com/lightning/settings/personal/ResetApiToken/home
with open("setup.yaml") as stream:
    try:
        #print(yaml.safe_load(stream))
        values = yaml.safe_load(stream)
        #print(values)
    except yaml.YAMLError as exc:
        pass
        print(exc)
#print(values)

In [3]:
import pandas as pd
import csv
import requests
from io import StringIO
from simple_salesforce import Salesforce
from getpass import getpass

#show all columns.... 
pd.set_option('display.max_columns', None)

# Input Salesforce credentials:
sf = Salesforce(
    username=values["username"], 
    password= getpass(),# gets input from Command prompt, hides what you type 
    security_token=values["security_token"])
print("Please wait while I extract that data for you")
# Basic report URL structure:
orgParams = 'https://tigertext.my.salesforce.com/' # you can see this in your Salesforce URL
exportParams = '?isdtp=p1&export=1&enc=UTF-8&xf=csv'

# Downloading the report:
#reportId = '00O5x000005FOpVEAW' # You find this in the URL of the report in question between "Report/" and "/view"
reportId = '00O5x000008GfBZEA0' # Added in Product Line column to address One-Time
reportUrl = orgParams + reportId + exportParams
reportReq = requests.get(reportUrl, headers=sf.headers, cookies={'sid': sf.session_id})
reportData = reportReq.content.decode('utf-8')
reportDf = pd.read_csv(StringIO(reportData))

········
Please wait while I extract that data for you


In [4]:
reportDf.drop(reportDf.tail(5).index, inplace = True) #Drop saleforce info junk lines at bottom
reportDf.loc[ reportDf["Account Category"] != "SMB", "Account Category"] = "Enterprise"
# Force 'close Date' to datetime, then add 'New Month' column with month (Kiets original excel file sets first day of month, instead of month. But both will be pivotable)
reportDf['Close Date'] = pd.to_datetime(reportDf['Close Date'], errors='coerce')
reportDf["New Month"] = reportDf["Close Date"].dt.month#.astype(str).str[0:2]
reportDf = reportDf[~reportDf['Opportunity Name'].str.contains('Test', na=False)]#remove test opps.. And TeleTest because f them that's why
#reportDf

In [5]:
# Removing Renwal's won 
reportDf = reportDf[~((reportDf["Opportunity Record Type"] == "Renewal") & (reportDf["Stage"] == "Won"))]


In [6]:
#New
#Upsell
#Cancelled
#Downgrade
#Cross-Sell

reportDf["Opp Type"] = "N/A"

reportDf.loc[ reportDf["Subtype"] == "New Business", "Opp Type"] = "New"
reportDf.loc[ reportDf["Subtype"] == "Upsell", "Opp Type"] = "Upsell"
reportDf.loc[ reportDf["Subtype"] == "Cross-Sell", "Opp Type"] = "Cross-Sell"
reportDf.loc[ reportDf["Subtype"] == "Down-Sell", "Opp Type"] = "Downgrade"
reportDf.loc[ reportDf["Subtype"] == "Cancellation", "Opp Type"] = "Cancelled"

reportDf#[0:40]
#reportDf.to_csv("Test.csv")

,Close Date,CASESAFEID,Ultimate Parent Account,Account Owner,Opportunity Owner,Partner*,Partner Margin (%),Parent Account,Account Name,Opportunity Name,Account Category,Type,Subtype,Opportunity Record Type,Stage,Product Code,Product Name,Quantity,Calculated Term (F),MRR Currency,MRR,Amount Currency,Amount,List Price Currency,List Price,Sales Price Currency,Sales Price,Total Price Currency,Total Price,One-Time Non-Implementation,Product Line,Product Line.1,Product Line 1,Product Line 2,Schedule Type,Category,MRR Currency.1,MRR.1,New Month,Opp Type
19,2023-07-19,0065x000022vQvCAAU,Erlanger Health System,Clint Fuller,Remy O'Donnell,No Partner,NaN,Erlanger Health System,Plastic Surgery Group/Chattanooga,Renewal - Plastic Surgery Group/Chattanooga - ...,Enterprise,Existing Business,Cancellation,Renewal,Lost,TTE - 3Y,TigerConnect Clinical Collaboration Platform –...,-32.0,36.0,USD,-176.0,USD,-6336.0,USD,6.0,USD,180.0,USD,-5760.0,0.0,TigerText Essentials,TigerText Essentials,NaN,NaN,Monthly,Subscription,USD,-160.0,7,Cancelled
20,2023-07-19,0065x000022vQvCAAU,Erlanger Health System,Clint Fuller,Remy O'Donnell,No Partner,NaN,Erlanger Health System,Plastic Surgery Group/Chattanooga,Renewal - Plastic Surgery Group/Chattanooga - ...,Enterprise,Existing Business,Cancellation,Renewal,Lost,TCAO - PSS,Premium Support,-32.0,36.0,USD,-176.0,USD,-6336.0,USD,1.0,USD,18.0,USD,-576.0,0.0,One-Time,One-Time,NaN,NaN,Monthly,Subscription,USD,-16.0,7,Cancelled
38,2023-07-21,0065x000023A7PsAAK,NaN,Edgar Castro,Jeff Hiller,No Partner,NaN,NaN,Nova Home Care LLC,Renewal - Nova Home Care LLC - 10/20/2023,SMB,Existing Business,Cancellation,Renewal,Lost,SMB - TTE - 1Y,SMB TigerConnect Clinical Collaboration Platfo...,-50.0,36.0,USD,-300.0,USD,-10800.0,USD,9.0,USD,216.0,USD,-10800.0,0.0,TigerText Essentials,TigerText Essentials,NaN,NaN,Monthly,Subscription,USD,-300.0,7,Cancelled
39,2023-07-13,0065x000023AJsqAAG,NaN,Paul Samson,Paul Samson,No Partner,NaN,NaN,Affiliated Dermatology,Renewal - Affiliated Dermatology - 9/11/2023,SMB,Existing Business,Cancellation,Renewal,Lost,TCAO - ATCH,TigerConnect®-Hosted Archiving,-150.0,36.0,USD,-150.0,USD,-5400.0,USD,1.0,USD,36.0,USD,-5400.0,0.0,Archiving,Archiving,NaN,NaN,Monthly,Subscription,USD,-150.0,7,Cancelled
42,2023-08-23,0065x000024U1gmAAC,NaN,Jeff Wu,Jeff Hiller,No Partner,NaN,NaN,The Home at Beth Sholom Village,Renewal - The Home at Beth Sholom Village - 2/...,SMB,Existing Business,Cancellation,Renewal,Lost,SMB - TTE - 1Y,SMB TigerConnect Clinical Collaboration Platfo...,-20.0,36.0,USD,-160.0,USD,-5760.0,USD,9.0,USD,252.0,USD,-5040.0,0.0,TigerText Essentials,TigerText Essentials,NaN,NaN,Monthly,Subscription,USD,-140.0,8,Cancelled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1137,2023-08-25,0065x00002BLYyzAAH,NaN,Jeff Hiller,Jeff Hiller,No Partner,NaN,NaN,Charles Nechtem Associates,Downgrade - Charles Nechtem Associates,SMB,Existing Business,Down-Sell,Amendment,Lost,SMB - TT PT MO EAP 3,TigerConnect Patient Engagement Pro,-4.0,12.0,USD,-196.0,USD,-2352.0,USD,40.0,USD,456.0,USD,-1824.0,0.0,TigerTouch+,TigerTouch+,NaN,NaN,Monthly,Subscription,USD,-152.0,8,Downgrade
1138,2023-08-25,0065x00002BLYyzAAH,NaN,Jeff Hiller,Jeff Hiller,No Partner,NaN,NaN,Charles Nechtem Associates,Downgrade - Charles Nechtem Associates,SMB,Existing Business,Down-Sell,Amendment,Lost,SMB - TTE - 1Y,SMB TigerConnect Clinical Collaboration Platfo...,-4.0,12.0,USD,-196.0,USD,-2352.0,USD,9.0,USD,120.0,USD,-480.0,0.0,TigerText Essentials,TigerText Essentials,NaN,NaN,Monthly,Subscription,USD,-40.0,8,Downgrade
1139,2023-08-25,0065x00002BLYyzAAH,NaN,Jeff Hiller,Jeff Hiller,No Partner,NaN,NaN,Charles Nechtem Associates,Downgrade - Charles Nechtem Associates,SMB,Existing Business,Down-Sell,Amendment,Lost,TCAO - ATCH,TigerConnect®-Hosted Archiving,-4.0,12.0,USD,-196.0,USD,-2352.0,USD,1.0,USD,12.0,USD,-48.0,0.0,Archiving,Archiving,NaN,NaN,Mo

In [7]:
# Add "One-Time?" Column based on Category
reportDf['One-Time?'] = reportDf['Category'].isin(['API', 'Subscription','Add-On'])
reportDf

,Close Date,CASESAFEID,Ultimate Parent Account,Account Owner,Opportunity Owner,Partner*,Partner Margin (%),Parent Account,Account Name,Opportunity Name,Account Category,Type,Subtype,Opportunity Record Type,Stage,Product Code,Product Name,Quantity,Calculated Term (F),MRR Currency,MRR,Amount Currency,Amount,List Price Currency,List Price,Sales Price Currency,Sales Price,Total Price Currency,Total Price,One-Time Non-Implementation,Product Line,Product Line.1,Product Line 1,Product Line 2,Schedule Type,Category,MRR Currency.1,MRR.1,New Month,Opp Type,One-Time?
19,2023-07-19,0065x000022vQvCAAU,Erlanger Health System,Clint Fuller,Remy O'Donnell,No Partner,NaN,Erlanger Health System,Plastic Surgery Group/Chattanooga,Renewal - Plastic Surgery Group/Chattanooga - ...,Enterprise,Existing Business,Cancellation,Renewal,Lost,TTE - 3Y,TigerConnect Clinical Collaboration Platform –...,-32.0,36.0,USD,-176.0,USD,-6336.0,USD,6.0,USD,180.0,USD,-5760.0,0.0,TigerText Essentials,TigerText Essentials,NaN,NaN,Monthly,Subscription,USD,-160.0,7,Cancelled,True
20,2023-07-19,0065x000022vQvCAAU,Erlanger Health System,Clint Fuller,Remy O'Donnell,No Partner,NaN,Erlanger Health System,Plastic Surgery Group/Chattanooga,Renewal - Plastic Surgery Group/Chattanooga - ...,Enterprise,Existing Business,Cancellation,Renewal,Lost,TCAO - PSS,Premium Support,-32.0,36.0,USD,-176.0,USD,-6336.0,USD,1.0,USD,18.0,USD,-576.0,0.0,One-Time,One-Time,NaN,NaN,Monthly,Subscription,USD,-16.0,7,Cancelled,True
38,2023-07-21,0065x000023A7PsAAK,NaN,Edgar Castro,Jeff Hiller,No Partner,NaN,NaN,Nova Home Care LLC,Renewal - Nova Home Care LLC - 10/20/2023,SMB,Existing Business,Cancellation,Renewal,Lost,SMB - TTE - 1Y,SMB TigerConnect Clinical Collaboration Platfo...,-50.0,36.0,USD,-300.0,USD,-10800.0,USD,9.0,USD,216.0,USD,-10800.0,0.0,TigerText Essentials,TigerText Essentials,NaN,NaN,Monthly,Subscription,USD,-300.0,7,Cancelled,True
39,2023-07-13,0065x000023AJsqAAG,NaN,Paul Samson,Paul Samson,No Partner,NaN,NaN,Affiliated Dermatology,Renewal - Affiliated Dermatology - 9/11/2023,SMB,Existing Business,Cancellation,Renewal,Lost,TCAO - ATCH,TigerConnect®-Hosted Archiving,-150.0,36.0,USD,-150.0,USD,-5400.0,USD,1.0,USD,36.0,USD,-5400.0,0.0,Archiving,Archiving,NaN,NaN,Monthly,Subscription,USD,-150.0,7,Cancelled,True
42,2023-08-23,0065x000024U1gmAAC,NaN,Jeff Wu,Jeff Hiller,No Partner,NaN,NaN,The Home at Beth Sholom Village,Renewal - The Home at Beth Sholom Village - 2/...,SMB,Existing Business,Cancellation,Renewal,Lost,SMB - TTE - 1Y,SMB TigerConnect Clinical Collaboration Platfo...,-20.0,36.0,USD,-160.0,USD,-5760.0,USD,9.0,USD,252.0,USD,-5040.0,0.0,TigerText Essentials,TigerText Essentials,NaN,NaN,Monthly,Subscription,USD,-140.0,8,Cancelled,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1137,2023-08-25,0065x00002BLYyzAAH,NaN,Jeff Hiller,Jeff Hiller,No Partner,NaN,NaN,Charles Nechtem Associates,Downgrade - Charles Nechtem Associates,SMB,Existing Business,Down-Sell,Amendment,Lost,SMB - TT PT MO EAP 3,TigerConnect Patient Engagement Pro,-4.0,12.0,USD,-196.0,USD,-2352.0,USD,40.0,USD,456.0,USD,-1824.0,0.0,TigerTouch+,TigerTouch+,NaN,NaN,Monthly,Subscription,USD,-152.0,8,Downgrade,True
1138,2023-08-25,0065x00002BLYyzAAH,NaN,Jeff Hiller,Jeff Hiller,No Partner,NaN,NaN,Charles Nechtem Associates,Downgrade - Charles Nechtem Associates,SMB,Existing Business,Down-Sell,Amendment,Lost,SMB - TTE - 1Y,SMB TigerConnect Clinical Collaboration Platfo...,-4.0,12.0,USD,-196.0,USD,-2352.0,USD,9.0,USD,120.0,USD,-480.0,0.0,TigerText Essentials,TigerText Essentials,NaN,NaN,Monthly,Subscription,USD,-40.0,8,Downgrade,True
1139,2023-08-25,0065x00002BLYyzAAH,NaN,Jeff Hiller,Jeff Hiller,No Partner,NaN,NaN,Charles Nechtem Associates,Downgrade - Charles Nechtem Associates,SMB,Existing Business,Down-Sell,Amendment,Lost,TCAO - ATCH,TigerConnect®-Hosted Archiving,-4.0,12.0,USD,-196.0,USD,-2352.0,USD,1.0,USD,

In [8]:
# creates "New MRR" Column
reportDf["New MRR"] = reportDf["Total Price"]/reportDf["Calculated Term (F)"]
#reportDf.loc[ reportDf["One-Time?"] == "Yes", "New MRR"] = 0 # One time payments don't have recurring revenue component. Changed from Yes/No to bool
reportDf.loc[ reportDf["One-Time?"] == False, "New MRR"] = 0 # The bool is flipped. Sue me.

In [9]:
#reportDf.to_csv("Test.csv")
#reportDf

In [10]:
# https://stackoverflow.com/questions/28236305/how-do-i-sum-values-in-a-column-that-match-a-given-condition-using-pandas
churn_errors = reportDf[reportDf['Opp Type'].isin(["Cancelled", "Downgrade"])]
churn_errors = churn_errors.groupby('Account Name')["New MRR"].sum()
# ^ Print here if you want full churn list
# May want > not >= below!
churn_errors = churn_errors[churn_errors >= 0]
churn_errors

Account Name
Rural Ob Access & Maternal Services (ROAMS)    0.0
Name: New MRR, dtype: float64

In [11]:
print("Fixing the following Churn errors: ")
print(churn_errors)
for i in churn_errors.index:
    print("           " + i + "       ...Fixed")
    # Can't be * -1, because it might have portions already negative. Honestly the whole churn check isn't perfect. 
    # might wanna revisit. You could have partial churn wrong if there's a large enough negative to cover it?
    # Maybe it's good. idk
    
    reportDf.loc[(reportDf['Account Name'] == i) & (reportDf['Opp Type'].isin(["Cancelled", "Downgrade"])) & (reportDf["New MRR"] > 0), "New MRR"] = reportDf["New MRR"] * -1
    # Can set recursive permission ask here. Or ask to send emails. 
# does this actually save the changes made? Error maybe idk gotta check this

Fixing the following Churn errors: 
Account Name
Rural Ob Access & Maternal Services (ROAMS)    0.0
Name: New MRR, dtype: float64
           Rural Ob Access & Maternal Services (ROAMS)       ...Fixed


In [12]:
#ramp_check = reportDf.groupby('Account Name')["New MRR"].sum()# - reportDf["MRR"]
#ramp_check =  reportDf.groupby('CASESAFEID')["New MRR"].transform('sum') - reportDf["MRR"]
# having ramp check ignore On-Time deals to make my life easier
ramp_check =  reportDf[reportDf["One-Time?"] == True].groupby('CASESAFEID')["New MRR"].transform('sum') - reportDf["MRR"]
ramp_check = ramp_check[ramp_check > 0.1]
#list(ramp_check.index)
#reportDf.loc[ramp_check.index]
ramp_check

909    54.0
910    54.0
911    54.0
912    54.0
dtype: float64

In [13]:
# Doesn't fix all issues. Best to write them to a fix manually section?

# Time to fix them? Idk if this works both ways. need to double check stuff
# Boolean mask for 'CASESAFEID's that are in 'ramp_check'
ramp_check_ids = reportDf.loc[ramp_check.index, 'CASESAFEID']
mask = reportDf['CASESAFEID'].isin(ramp_check_ids)


# Get calculated Term corrected
reportDf['Amount'] = reportDf.groupby(['CASESAFEID'])['MRR.1'].transform('sum') #sum of product MRR?
reportDf['Calculated Term (F)'] = round(reportDf['Amount'] / reportDf['MRR']) #think this is wrong, only works if length is equal for all portions of the ramp deal.


# Calculate the total "New MRR" for each 'CASESAFEID' that are in 'ramp_check'
#total_new_mrr = reportDf[mask].groupby('CASESAFEID')["New MRR"].transform('sum') # need to divide by calculated term. But it can be annual so 1 is 12. There's another field we can use i think
#total_new_mrr = reportDf[mask].groupby(['CASESAFEID', 'Product Name'])["MRR"].transform(lambda x: x.iloc[0]) # using transform because index [0] won't work. f pandas
# New MRR is messy because calculated term can be WRONG. Fucking SF data. Soooo... We're gonna compare and get a %
# of 


#total_new_mrr = reportDf[mask].groupby(['CASESAFEID', 'Product Name'])["MRR"].transform(lambda x: x.iloc[0])
#total_new_mrr = reportDf[mask].groupby(['CASESAFEID', 'Product Name'])["MRR.1"].transform('sum')
total_new_mrr = reportDf[mask].groupby(['CASESAFEID', 'Product Name'])["MRR.1"].transform('sum') / reportDf[mask]['Calculated Term (F)']


# Find the first occurrence of each 'CASESAFEID' that are in 'ramp_check'
#is_first_occurrence = ~reportDf[mask].duplicated('CASESAFEID')

# If it is the first occurrence, set the value to the total, otherwise set to 0
#reportDf.loc[mask, 'New MRR'] = np.where(is_first_occurrence, total_new_mrr, 0)
is_first_occurrence = ~reportDf[mask].duplicated(['CASESAFEID', 'Product Name'])
reportDf.loc[mask, 'New MRR'] = np.where(is_first_occurrence, total_new_mrr, 0)

reportDf[reportDf["Account Name"] == "AIDS Healthcare Foundation"] #testing, viewing specific ramp deal.

,Close Date,CASESAFEID,Ultimate Parent Account,Account Owner,Opportunity Owner,Partner*,Partner Margin (%),Parent Account,Account Name,Opportunity Name,Account Category,Type,Subtype,Opportunity Record Type,Stage,Product Code,Product Name,Quantity,Calculated Term (F),MRR Currency,MRR,Amount Currency,Amount,List Price Currency,List Price,Sales Price Currency,Sales Price,Total Price Currency,Total Price,One-Time Non-Implementation,Product Line,Product Line.1,Product Line 1,Product Line 2,Schedule Type,Category,MRR Currency.1,MRR.1,New Month,Opp Type,One-Time?,New MRR
807,2023-08-10,0065x00002BKi3fAAD,NaN,Samantha Bailey,Samantha Bailey,No Partner,NaN,NaN,AIDS Healthcare Foundation,CSQL - AIDS Healthcare Foudation - Batch Upload,Enterprise,Existing Business,Cross-Sell,SMB Sales,Won,PS - TS,Technical Services,4.0,1.0,USD,208.33,USD,208.33,USD,225.00,USD,115.00,USD,460.00,0.0,One-Time,One-Time,Add-on,NaN,No Schedule,Implementation,USD,0.00,8,Cross-Sell,False,0.00
808,2023-08-10,0065x00002BKi3fAAD,NaN,Samantha Bailey,Samantha Bailey,No Partner,NaN,NaN,AIDS Healthcare Foundation,CSQL - AIDS Healthcare Foudation - Batch Upload,Enterprise,Existing Business,Cross-Sell,SMB Sales,Won,TCAO - FSINT,Batch Upload Service,1.0,1.0,USD,208.33,USD,208.33,USD,208.33,USD,2499.96,USD,2499.96,0.0,Integrations,Integrations,Add-on,NaN,Monthly,Subscription,USD,208.33,8,Cross-Sell,True,208.33
1054,2023-08-11,0065x00002BLEB1AAP,NaN,Samantha Bailey,Tony Creel,No Partner,NaN,NaN,AIDS Healthcare Foundation,Downgrade - AIDS Healthcare Foundation,Enterprise,Existing Business,Down-Sell,Amendment,Lost,TCAO - RDINT,Redox EHR Integration,-1.0,1.0,USD,-750.00,USD,-750.00,USD,833.33,USD,9000.00,USD,-9000.00,0.0,Integrations,Integrations,NaN,NaN,Monthly,Subscription,USD,-750.00,8,Downgrade,True,-750.00


In [14]:
#reportDf = reportDf.drop(columns=["One-Time?"])
reportDf.to_excel("bookings_output.xlsx")  
#from openpyxl import load_workbook

#book = load_workbook("Q4 Sales Report test.xlsx")
#writer = pd.ExcelWriter("Q4 Sales Report test.xlsx", engine = 'openpyxl')
#writer.book = book

#reportDf.to_excel(writer, sheet_name = 'SF Report')
#writer.save()
#writer.close()
#reportDf.to_excel("Q4 Sales Report test.xlsx")  

In [15]:
# https://stackoverflow.com/questions/71132809/valueerror-grouper-for-xxx-not-1-dimensional-with-pandas-pivot-table
table = reportDf[reportDf['Opp Type'].isin(["New", "Upsell", "Cross-Sell"])].pivot_table(values='New MRR', index='Opportunity Owner', aggfunc=np.sum, fill_value=0, margins = True)
table

,New MRR
Opportunity Owner,
Amy Peyton,0.000000
Andrew Cordell,150.000000
Basya Kussner,0.000000
Benjamin Lee,385.000000
Bob Cooper,9475.000000
Carol Jarosky,0.000000
Casey Sullivan,0.000000
Chase Wienckowski,3754.000000
Clint Fuller,7011.730000


In [16]:
churn_table = reportDf[reportDf['Opp Type'].isin(["Downgrade", "Cancelled"])].pivot_table(values='New MRR', index='Opportunity Owner', aggfunc=np.sum, fill_value=0, margins = True)

table = table.merge(churn_table, on='Opportunity Owner', how='outer' )

# Remove "All" row and substitute with Total. Just cosmetic
table = table.drop("All", axis=0)
table.loc['Total'] = table.sum(numeric_only=True)
table = round(table,0)#loc[["All"]].index

In [17]:
# Some issue with TCAO - APIAF it doesn't have a product line? stupid salesforce data
reportDf.loc[ reportDf["Product Line 1"].isna(), "Product Line 1"] = reportDf['Product Line'] #handles the broken blank lines in https://tigertext.lightning.force.com/lightning/o/Product2/list?filterName=00B600000061NrGEAU
reportDf.loc[ reportDf["Product Line 1"].isna(), "Product Line 1"] = 'CCP Standard'
#Try without the above 2 lines, figure out what the difference between yours and kiet's product pivot is.
# Kiet's (running total dash, sales report) product line is based on Product Code, and put through his mapping.
# you can pull the product code report and cross reference his mapping to see what the best match is. 

# PE
reportDf.loc[ reportDf["Product Line 1"] == "Patient Engagement", "Product Line 1"] = "Patient Engagement"
reportDf.loc[ reportDf["Product Line 1"] == "Patient Engagement Pro", "Product Line 1"] = "Patient Engagement"
reportDf.loc[ reportDf["Product Line 1"] == "TigerTouch", "Product Line 1"] = "Patient Engagement"
reportDf.loc[ reportDf["Product Line 1"] == "TigerTouch+", "Product Line 1"] = "Patient Engagement"

# CCP Standard
#reportDf.loc[ reportDf["Product Line 1"] == "Add-on", "Product Line 1"] = "CCP Standard"
reportDf.loc[ reportDf["Product Line 1"] == "CCP Standard", "Product Line 1"] = "CCP Standard"
reportDf.loc[ reportDf["Product Line 1"] == "One-Time", "Product Line 1"] = "CCP Standard"
reportDf.loc[ reportDf["Product Line 1"] == "TigerText Essentials", "Product Line 1"] = "CCP Standard"
#reportDf.loc[ reportDf["Product Line 1"] == "VoIP", "Product Line 1"] = "CCP Standard"

# CCP (Pro & Advanced)
reportDf.loc[ reportDf["Product Line 1"] == "CCP Advanced", "Product Line 1"] = "CCP (Pro & Advanced)"
reportDf.loc[ reportDf["Product Line 1"] == "CCP Pro", "Product Line 1"] = "CCP (Pro & Advanced)"
reportDf.loc[ reportDf["Product Line 1"] == "TigerFlow", "Product Line 1"] = "CCP (Pro & Advanced)"
reportDf.loc[ reportDf["Product Line 1"] == "TigerFlow+", "Product Line 1"] = "CCP (Pro & Advanced)"

# Schedule
reportDf.loc[ reportDf["Product Line 1"] == "Physician Scheduling", "Product Line 1"] = "Physician Scheduling"
reportDf.loc[ reportDf["Product Line 1"] == "Resident Scheduling", "Product Line 1"] = "Physician Scheduling"
reportDf.loc[ reportDf["Product Line 1"] == "TigerSchedule", "Product Line 1"] = "Physician Scheduling"

# Middleware
reportDf.loc[ reportDf["Product Line 1"] == "Middleware", "Product Line 1"] = "Middleware"

# Add-On, this line doesn't do anything, just a reminder. 
reportDf.loc[ reportDf["Product Line 1"] == "Add-on", "Product Line 1"] = "Add-on"
reportDf.loc[ reportDf["Product Line 1"] == "VoIP", "Product Line 1"] = "VoIP"

#productPivot = reportDf[reportDf['Opp Type'].isin(["New", "Upsell", "Cross-Sell"])].pivot_table(values='New MRR', index='Product Line 1', aggfunc=np.sum, fill_value=0, margins = True)
#productPivot
productPivot = reportDf[reportDf['Opp Type'].isin(["New", "Upsell", "Cross-Sell"])].pivot_table(
    values='New MRR', 
    index='Product Line 1', 
    columns='Account Category', 
    aggfunc=np.sum, 
    fill_value=0, 
    margins=True
)

# Print the pivot table
productPivot = productPivot.round(0)
productPivot


Account Category,Enterprise,SMB,All
Product Line 1,,,
Add-on,25708.0,0.0,25708.0
Archiving,0.0,401.0,401.0
CCP (Pro & Advanced),38895.0,991.0,39886.0
CCP Standard,6352.0,15025.0,21376.0
Integrations,-4.0,0.0,-4.0
Patient Engagement,560.0,6952.0,7512.0
Physician Scheduling,536.0,0.0,536.0
VoIP,0.0,399.0,399.0
All,72048.0,23768.0,95816.0


In [18]:
dfi.export(
    table,
    "table.png",
    table_conversion="matplotlib"
)

In [19]:
today = datetime.today()
date1 = today.strftime('%y%m%d')
print(date1)

230905


In [20]:
reportExportFormat = reportDf

book = load_workbook('DONOTTOUCH.xlsx')

import shutil

# copy original file
shutil.copy('DONOTTOUCH.xlsx', date1 + ' Weekly Sales Report.xlsx')
book = load_workbook('DONOTTOUCH.xlsx')

sheet = book['SF']
rows = dataframe_to_rows(reportDf, index=False, header=True)
for i, row in enumerate(rows, 1):
    for j, value in enumerate(row, 1):
        sheet.cell(row=i, column=j, value=value)
        
sheet = book['SF Sales Complete']
rows = dataframe_to_rows(reportDf, index=False, header=True)
for i, row in enumerate(rows, 1):
    for j, value in enumerate(row, 1):
        sheet.cell(row=i, column=j, value=value)

# Save the workbook
book.save(date1 + ' Weekly Sales Report.xlsx')

In [21]:
print("Have a wonderful day!")
print("       .-.")
print(" __   /   \   __")
print("(  `\'.\   /.\'`  )")
print(" \'-._.(;;;)._.-\'")
print(" .-\'  ,`\"`,  \'-.")
print("(__.-\'/   \'-.__)/)_")
print("      \   /\    / / )")
print("       \'-\'  |   \/.-\')")
print("       ,    | .\'/\\'..)")
print("       |\   |/  | \_)")
print("       \ |  |   \_/")
print("        | \ /")
print("         \|/    _,")
print("          /  __/ /")
print("         | _/ _.\'")
print("         |/__/")
print("          \ ")

Have a wonderful day!
       .-.
 __   /   \   __
(  `'.\   /.'`  )
 '-._.(;;;)._.-'
 .-'  ,`"`,  '-.
(__.-'/   '-.__)/)_
      \   /\    / / )
       '-'  |   \/.-')
       ,    | .'/\'..)
       |\   |/  | \_)
       \ |  |   \_/
        | \ /
         \|/    _,
          /  __/ /
         | _/ _.'
         |/__/
          \ 


In [22]:
#input("") #Keeps terminal open until you press enter. But kinda bad practice
import os 
os.system("pause")

0